#Transfer Learning and Pre-trained Models

##Transfer Learning

Transfer learning is a popular method in deep learning because it allows us to build accurate models in a time-saving manner. With transfer learning, instead of starting the learning process from scratch, you start from patterns that have been learned when solving a different problem. This way you leverage previous learnings and avoid starting from scratch.

###Animal species identification

In this example, we will use a pre-trained model from Keras (InceptionV3) to do animal species identification. We'll assume we're working with a dataset of various animal species.

Please install necessary libraries: pip install tensorflow pandas matplotlib.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%%capture
!pip install numpy tensorflow matplotlib

First, we import all necessary libraries:

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

Let's download the pre-trained InceptionV3 model (include_top is set to False in order to discard the last (top) layer of the model, which is responsible for classification. We'll then add our own classification layer for the animal species.)

In [ ]:
# Download the inception v3 weights
base_model = InceptionV3(weights='imagenet', include_top=False)

87910968/87910968 [==============================] - 1s 0us/step


Now we'll add a global spatial average pooling layer, and a fully-connected (dense) layer that will output the predicted probability for each class:

In [ ]:
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# add a fully-connected layer
x = Dense(1024, activation='relu')(x)

# and a logistic layer with the number of classes (assuming we have 3 classes)
predictions = Dense(3, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

We're now going to freeze the layers of the Inception V3 model and only train the last layers we added:

In [ ]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

We'll use ImageDataGenerator for loading and transforming our image dataset:

In [ ]:
# assuming your data is in 'data/train' and 'data/validation' directories
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/lab/Presentations/AI_summer/data/animal_classification/train',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/lab/Presentations/AI_summer/data/animal_classification/validation',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical')

Found 360 images belonging to 3 classes.
Found 90 images belonging to 3 classes.


Finally, we can start training our model:

In [ ]:
# train the model
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator)

In terms of application, the model trained here can be used to automatically classify images of animals into their respective species. This can be useful in a wide range of contexts, such as in wildlife research to automate the process of species identification, or in pet adoption centers to help categorize animals. The concept of transfer learning applied here is also a very versatile technique and can be used in various other applications such as object detection, more complex image or scene recognition tasks, or even in NLP (Natural Language Processing).

###Language translation

Language translation is a common application of machine learning, specifically the field of Natural Language Processing (NLP). An effective approach to building a language translation model involves the use of sequence-to-sequence (seq2seq) models, which are a type of Recurrent Neural Network (RNN). These models are beneficial because they allow for variable input and output sizes.



We’ll walk through the process of using a pre-trained translation model from the Hugging Face library in Google Colab. We’ll use the MarianMTModel, a state-of-the-art multilingual translation model that supports many languages. By the end of this guide, you’ll be able to translate text from English to various languages, such as Spanish, French, Russian, and Mandarin.

MarianMTModel is a family of neural machine translation models trained on the OPUS dataset, a large-scale multilingual corpus. These models are designed for translating text between multiple languages, and they achieve state-of-the-art performance in many translation tasks. The MarianMTModel is named after the Marian NMT framework, an efficient C++ implementation of neural machine translation that is used for research purposes.

In [ ]:
#Install the HuggingFace Transformers Library
%%capture
!pip install transformers

In [ ]:
#Install SentencePiece Library
%%capture
!pip install sentencepiece

In [ ]:
#Import required libraries
from transformers import MarianMTModel, MarianTokenizer

In [ ]:
#Define Translation Function (Create a function to translate text from English to a target language using a pre-trained model)
def translate(text, target_language):
    model_name = f'Helsinki-NLP/opus-mt-en-{target_language}'
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name)

    inputs = tokenizer.encode(text, return_tensors="pt")
    outputs = model.generate(inputs, num_beams=4, max_length=50, early_stopping=True)
    translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return translated_text

Use the translation function to translate text from English to the desired target language. Replace 'es' with the language code of your choice (e.g., 'fr' for French, 'ru' for Russian, or 'zh' for Mandarin):

In [ ]:
#Translate Text
input_text = "Hello, how are you?"
target_language = 'es'
translated_text = translate(input_text, target_language)
print(translated_text)

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Hola, ¿cómo estás?


##Fine tuning pre-trained models

Fine-tuning pre-trained models can be accomplished in a few ways. Here are the three most common approaches:
>Full Network Fine-Tuning: Fine-tune all layers in the pre-trained model. In this approach, you use the pre-trained model and adjust its weights based on your data using backpropagation and a low learning rate. This method can often yield superior results when you have a large amount of data.

>Feature Extraction: Keep all layers in the pre-trained model fixed, and only train the final layer(s) that are responsible for the task at hand (e.g., classification). This approach is faster and can be useful when you don't have much data.

>Layer-wise Fine-Tuning: Fine-tune only a portion of the layers in the pre-trained model. This is a middle ground between full network fine-tuning and feature extraction. You might want to keep early layers fixed (since they capture more generic features) and fine-tune some higher layers (which capture more task-specific features).

####Full Network Fine-Tuning

The script fully trains the model on the CIFAR-10 dataset

In [ ]:
import torch
from torchvision import datasets, models, transforms
from torch import nn, optim

In [ ]:
# Load the CIFAR-10 dataset
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [ ]:
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

100%|██████████| 170498071/170498071 [00:04<00:00, 40772074.53it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


In [ ]:
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

Files already downloaded and verified


In [ ]:
# Define the pre-trained model
model = models.resnet50(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 74.6MB/s]


In [ ]:
# Make sure all parameters are being tracked for gradients and updated during training
for param in model.parameters():
    param.requires_grad = True

In [ ]:
# Modify the final layer to match the number of classes in CIFAR-10 (10 classes)
model.fc = nn.Linear(2048, 10)

In [ ]:
# Move the model to GPU if available
%%capture
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
# Training loop
for epoch in range(2):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

Finished Training


#### Feature Extraction

In feature extraction, we freeze all the layers except the final one. We change the final layer to match our problem's requirements and only train the final layer.

In [ ]:
import torch
from torchvision import datasets, models, transforms
from torch import nn, optim

In [ ]:
# Load the CIFAR-10 dataset
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [ ]:
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

In [ ]:
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

In [ ]:
# Define the pre-trained model
model = models.resnet50(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
# Freeze all layers
for param in model.parameters():
    param.requires_grad = False

In [ ]:
# Modify the final layer to match the number of classes in CIFAR-10 (10 classes)
model.fc = nn.Linear(2048, 10)

In [ ]:
# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)

In [ ]:
# Training loop
for epoch in range(2):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

Finished Training


####Layer-wise Fine-Tuning

In layer-wise fine-tuning, we gradually unfreeze layers from the end, i.e., first only train the last few layers, then train more layers, and so on.In this layer-wise fine-tuning example, we start by only training the final layer. After that, we unfreeze the next layer (in this case, layer4, which is the last layer before the fully connected layer in the ResNet model). You can continue this process, unfreezing more layers and continuing training as needed for your specific application.

In [ ]:
import torch
from torchvision import datasets, models, transforms
from torch import nn, optim

In [ ]:
# Load the CIFAR-10 dataset
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [ ]:
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

In [ ]:
# Define the pre-trained model
model = models.resnet50(pretrained=True)

In [ ]:
# Freeze all layers initially
for param in model.parameters():
    param.requires_grad = False

In [ ]:
# Unfreeze the last layer
for param in model.fc.parameters():
    param.requires_grad = True

In [ ]:
# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001, momentum=0.9)

In [ ]:
# Training loop
for epoch in range(2):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

In [ ]:
# Now unfreeze the next layer and update optimizer
for param in model.layer4.parameters():
    param.requires_grad = True

In [ ]:
optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001, momentum=0.9)

In [ ]:
# Continue training
# ...

print('Finished Training')

###Emotion Recognition in images

Emotion recognition is a task that requires a classification model. The FER2013 dataset is a commonly used dataset for this task, which is available on Kaggle. It contains 48x48 pixel grayscale images of faces, with labels for seven emotions (Angry, Disgust, Fear, Happy, Sad, Surprise, Neutral). We can fine-tune a pre-trained model such as ResNet for this task.

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('default')

import os
import tensorflow as tf
import keras
import cv2

from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model
from tensorflow.keras import layers , models, optimizers

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import *
from tensorflow.keras.applications import ResNet50V2

Visualizing the classes

In [ ]:
train_dir = '/content/drive/MyDrive/lab/Presentations/AI_summer/data/fer2013/train/'
test_dir = '/content/drive/MyDrive/lab/Presentations/AI_summer/data/fer2013/test/'

def Classes_Count( path, name):
    Classes_Dict = {}

    for Class in os.listdir(path):

        Full_Path = path + Class
        Classes_Dict[Class] = len(os.listdir(Full_Path))

    df = pd.DataFrame(Classes_Dict, index=[name])

    return df

Train_Count = Classes_Count(train_dir, 'Train').transpose().sort_values(by="Train", ascending=False)
Test_Count = Classes_Count(test_dir, 'Test').transpose().sort_values(by="Test", ascending=False)

In [ ]:
pd.concat([Train_Count,Test_Count] , axis=1)

In [ ]:
Train_Count.plot(kind='barh')

In [ ]:
Test_Count.plot(kind='barh')

In [ ]:
plt.style.use('default')
plt.figure(figsize = (25, 8))
image_count = 1
BASE_URL = '/content/drive/MyDrive/lab/Presentations/AI_summer/data/fer2013/train/'

for directory in os.listdir(BASE_URL):
    if directory[0] != '.':
        for i, file in enumerate(os.listdir(BASE_URL + directory)):
            if i == 1:
                break
            else:
                fig = plt.subplot(1, 7, image_count)
                image_count += 1
                image = cv2.imread(BASE_URL + directory + '/' + file)
                plt.imshow(image)
                plt.title(directory, fontsize = 20)

Data preprocessing

In [ ]:
img_shape = 48
batch_size = 64
train_data_path ='/content/drive/MyDrive/lab/Presentations/AI_summer/data/fer2013/train/'
test_data_path = '/content/drive/MyDrive/lab/Presentations/AI_summer/data/fer2013/test/'

In [ ]:
train_preprocessor = ImageDataGenerator(
        rescale = 1 / 255.,
        # Data Augmentation
        rotation_range=10,
        zoom_range=0.2,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,
        fill_mode='nearest',
    )

In [ ]:
test_preprocessor = ImageDataGenerator(
    rescale = 1 / 255.,
)

In [ ]:
train_data = train_preprocessor.flow_from_directory(
    train_data_path,
    class_mode="categorical",
    target_size=(img_shape,img_shape),
    color_mode='rgb',
    shuffle=True,
    batch_size=batch_size,
    subset='training',
)

In [ ]:
test_data = test_preprocessor.flow_from_directory(
    test_data_path,
    class_mode="categorical",
    target_size=(img_shape,img_shape),
    color_mode="rgb",
    shuffle=False,
    batch_size=batch_size,
)

ResNet50V2 model

In [ ]:
# specifing new image shape for resnet
img_shape = 224
batch_size = 64
train_data_path ='/content/drive/MyDrive/lab/Presentations/AI_summer/data/fer2013/train/'
test_data_path = '/content/drive/MyDrive/lab/Presentations/AI_summer/data/fer2013/test/'

In [ ]:
train_preprocessor = ImageDataGenerator(
        rescale = 1 / 255.,
        rotation_range=10,
        zoom_range=0.2,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,
        fill_mode='nearest',
    )

test_preprocessor = ImageDataGenerator(
    rescale = 1 / 255.,
)

train_data = train_preprocessor.flow_from_directory(
    train_data_path,
    class_mode="categorical",
    target_size=(img_shape,img_shape),
    color_mode='rgb',
    shuffle=True,
    batch_size=batch_size,
    subset='training',
)

test_data = test_preprocessor.flow_from_directory(
    test_data_path,
    class_mode="categorical",
    target_size=(img_shape,img_shape),
    color_mode="rgb",
    shuffle=False,
    batch_size=batch_size,
)

Fine-tuning ResNet50V2

In [ ]:
ResNet50V2 = tf.keras.applications.ResNet50V2(input_shape=(224, 224, 3),
                                               include_top= False,
                                               weights='imagenet'
                                               )

In [ ]:
ResNet50V2.summary()

In [ ]:
# Freezing all layers except last 50

ResNet50V2.trainable = True

for layer in ResNet50V2.layers[:-50]:
    layer.trainable = False

In [ ]:
def Create_ResNet50V2_Model():

    model = Sequential([
                      ResNet50V2,
                      Dropout(.25),
                      BatchNormalization(),
                      Flatten(),
                      Dense(64, activation='relu'),
                      BatchNormalization(),
                      Dropout(.5),
                      Dense(7,activation='softmax')
                    ])
    return model

In [ ]:
ResNet50V2_Model = Create_ResNet50V2_Model()

ResNet50V2_Model.summary()

ResNet50V2_Model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Specifying Callbacks

In [ ]:
# Create Callback Checkpoint
checkpoint_path = "ResNet50V2_Model_Checkpoint"

Checkpoint = ModelCheckpoint(checkpoint_path, monitor="val_accuracy", save_best_only=True)

# Create Early Stopping Callback to monitor the accuracy
Early_Stopping = EarlyStopping(monitor = 'val_accuracy', patience = 7, restore_best_weights = True, verbose=1)

# Create ReduceLROnPlateau Callback to reduce overfitting by decreasing learning
Reducing_LR = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                  factor=0.2,
                                                  patience=2,
#                                                   min_lr=0.00005,
                                                  verbose=1)

callbacks = [Early_Stopping, Reducing_LR]

In [ ]:
steps_per_epoch = train_data.n // train_data.batch_size
validation_steps = test_data.n // test_data.batch_size

In [ ]:
ResNet50V2_history = ResNet50V2_Model.fit(
    train_data ,
    validation_data = test_data ,
    epochs=30,
    batch_size=batch_size,
    callbacks = callbacks,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps)

Evaluating ResNet50V2

In [ ]:
ResNet50V2_Score = ResNet50V2_Model.evaluate(test_data)

print("    Test Loss: {:.5f}".format(ResNet50V2_Score[0]))
print("Test Accuracy: {:.2f}%".format(ResNet50V2_Score[1] )

In [ ]:
def plot_curves(history):

    loss = history.history["loss"]
    val_loss = history.history["val_loss"]

    accuracy = history.history["accuracy"]
    val_accuracy = history.history["val_accuracy"]

    epochs = range(len(history.history["loss"]))

    plt.figure(figsize=(15,5))

    #plot loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, loss, label = "training_loss")
    plt.plot(epochs, val_loss, label = "val_loss")
    plt.title("Loss")
    plt.xlabel("epochs")
    plt.legend()

    #plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, accuracy, label = "training_accuracy")
    plt.plot(epochs, val_accuracy, label = "val_accuracy")
    plt.title("Accuracy")
    plt.xlabel("epochs")
    plt.legend()

In [ ]:
plot_curves(ResNet50V2_history)

In [ ]:
ResNet50V2_Predictions = ResNet50V2_Model.predict(test_data)

# Choosing highest probalbilty class in every prediction
ResNet50V2_Predictions = np.argmax(ResNet50V2_Predictions, axis=1)

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix

In [ ]:
fig , ax= plt.subplots(figsize=(15,10))

cm=confusion_matrix(test_data.labels, ResNet50V2_Predictions)

sns.heatmap(cm, annot=True, fmt='g', ax=ax)

ax.set_xlabel('Predicted labels',fontsize=15, fontweight='bold')
ax.set_ylabel('True labels', fontsize=15, fontweight='bold')
ax.set_title('ResNet50V2 Confusion Matrix', fontsize=20, fontweight='bold')

Visualizing predictions

In [ ]:
Emotion_Classes = ['Angry',
                  'Disgust',
                  'Fear',
                  'Happy',
                  'Neutral',
                  'Sad',
                  'Surprise']

In [ ]:
# Shuffling Test Data to show diffrent classes
test_preprocessor = ImageDataGenerator(
        rescale = 1 / 255.,
    )

In [ ]:
test_generator = test_preprocessor.flow_from_directory(
    test_data_path,
    class_mode="categorical",
    target_size=(img_shape,img_shape),
    color_mode="rgb",
    shuffle=True,
    batch_size=batch_size,
)

In [ ]:
# Display 10 random pictures from the dataset with their labels

Random_batch = np.random.randint(0, len(test_generator) - 1)

Random_Img_Index = np.random.randint(0, batch_size - 1 , 10)

fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(25, 10),
                        subplot_kw={'xticks': [], 'yticks': []})

for i, ax in enumerate(axes.flat):

    Random_Img = test_generator[Random_batch][0][Random_Img_Index[i]]

    Random_Img_Label = np.argmax(test_generator[Random_batch][1][Random_Img_Index[i]])

    Model_Prediction = np.argmax(ResNet50V2_Model.predict( tf.expand_dims(Random_Img, axis=0) , verbose=0))

    ax.imshow(Random_Img)

    if Emotion_Classes[Random_Img_Label] == Emotion_Classes[Model_Prediction]:
          color = "green"
    else:
          color = "red"
    ax.set_title(f"True: {Emotion_Classes[Random_Img_Label]}\nPredicted: {Emotion_Classes[Model_Prediction]}", color=color)
plt.show()
plt.tight_layout()

Predicting new images (Downloading OpenCV haarcascade frontalface Detection)

In [ ]:
!wget https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml

faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

In [ ]:
def load_and_prep_image(filename, img_shape = 224):

    img = cv2.imread(filename)

    GrayImg = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    faces = faceCascade.detectMultiScale(GrayImg, 1.1, 4)

    for x,y,w,h in faces:

        roi_GrayImg = GrayImg[ y: y + h , x: x + w ]
        roi_Img = img[ y: y + h , x: x + w ]

        cv2.rectangle(img, (x,y), (x+w, y+h), (0, 255, 0), 2)

        plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))

        faces = faceCascade.detectMultiScale(roi_Img, 1.1, 4)

        if len(faces) == 0:
            print("No Faces Detected")
        else:
            for (ex, ey, ew, eh) in faces:
                img = roi_Img[ ey: ey+eh , ex: ex+ew ]

    RGBImg = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

    RGBImg= cv2.resize(RGBImg,(img_shape,img_shape))

    RGBImg = RGBImg/255.

    return RGBImg

In [ ]:
def pred_and_plot(filename, class_names):

    # Import the target image and preprocess it
    img = load_and_prep_image(filename)

    # Make a prediction
    pred = ResNet50V2_Model.predict(np.expand_dims(img, axis=0))

    # Get the predicted class
    pred_class = class_names[pred.argmax()]

    # Plot the image and predicted class
    #plt.imshow(img)
    plt.title(f"Prediction: {pred_class}")
    plt.axis(False);

In [ ]:
pred_and_plot("/kaggle/input/fer2013/test/fear/PrivateTest_11014592.jpg", Emotion_Classes) # with ResNet50V2

##Pre-trained models for natural language understanding

###Chatbot development

We will use the transformer-based model BERT for our application, and the MultiWOZ dataset (a collection of multi-domain goal-oriented dialogues) for training.

In this script, we first load the MultiWOZ dataset and prepare it for training. The BERT tokenizer is used to encode the text data into a format that can be fed into the model. We then fine-tune the pre-trained BERT model on our dataset in a training loop. Please ensure that the MultiWOZ dataset is preprocessed into a suitable format where each row contains a dialogue turn and its corresponding label.

In [ ]:
import torch
from transformers import BertForSequenceClassification, BertTokenizer, AdamW
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np

In [ ]:
# Load the MultiWOZ dataset
# The dataset should be preprocessed into a format where each row contains a dialogue turn and its corresponding label
class MultiWOZDataset(Dataset):
    def __init__(self, data_file, tokenizer, max_length=512):
        self.data = pd.read_csv(data_file)
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        text = row['text']
        label = row['label']

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

In [ ]:
# Initialize the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

In [ ]:
# Load the MultiWOZ dataset
data_file = 'multiwoz.csv'  # Change to the path of your dataset
dataset = MultiWOZDataset(data_file, tokenizer)
dataloader = DataLoader(dataset, batch_size=16)

In [ ]:
# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
# Define loss function and optimizer
loss_fn = torch.nn.CrossEntropyLoss().to(device)
optimizer = AdamW(model.parameters(), lr=1e-5)

In [ ]:
# Training loop
for epoch in range(2):
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        optimizer.zero_grad()

    print(f'Epoch {epoch+1}/{2} loss: {loss.item()}')

print('Finished Training')